# Logic 

* What do words mean? 
* Propositional logic
* First-order logic
* Representing sentential meaning with logic
* Ontologies, Knowledge bases, and the Semantic Web
* Advanced topics: Lambda Calculus, representing events

## What do words mean?

* Dictionary definitions?
* Lexical categories?
* Relationship to other words?
* Context in which they appear?
* Referents in the "real world"?
* Effects in the mind of a listener?

## Propositional Logic

**Logic** is a system for deciding whether a statement is true or false. One basic version of it, known as [propositional logic](https://en.wikipedia.org/wiki/Propositional_calculus), is analogous to the system of boolean operators (ie, AND, OR, etc.) found in programming languages like Python, and operates in the same way. A proposition can be decided to be True or False based on the application of boolean operators to atomic propositions (often denoted with symbols such as *P*, *Q*, and *R*).

**Atomic propositions** are simple statements that evaluate to "True" or "False", and don't contain any complex operators (like AND, OR, ALL) - but we can construct more complex statements from them.

For example, if we have two atomic statements: **P** = "Garrett likes ice cream." and  **Q** = "Garrett likes chocolate.", then R = P AND Q = "Garrett likes ice cream and Garrett likes chocolate." should also evaluate to True. (TRUE AND TRUE evaluates to TRUE)

We'll be using a module for logic included in NLTK. It's not really suitable for serious logical inference, but it'll suffice for our needs here. Let's first look at the basic operators in propositional logic.  **Operators** are used to combine atomic values into complex logical statements.

In [1]:
import nltk
nltk.boolean_ops()

negation       	-
conjunction    	&
disjunction    	|
implication    	->
equivalence    	<->


Negation, conjunction, and disjunction hopefully need no introduction; they are just the equivalent of Python `not`, `and`, and `or`. Let's see how these work. We will need to create a Model which has a particular truth **Valuation** for atomic propositions. 

In [2]:
from nltk import Valuation, Assignment, Model

val = Valuation([('P', True), ('Q', True), ('R', False)]) #Just 3 statements
dom = set() #Domain is empty - coming up later
g = Assignment(dom)
m = Model(dom, val)

Given the Truth values of P, Q, and R given in `val` above, we can use the `evaluate` method of the model to get the truth value of more complex propositions

In [3]:
m.evaluate('-P',g) # not True should be False

False

In [4]:
m.evaluate('-R',g) # not False should be True

True

In [5]:
m.evaluate('P & Q',g) # True and True should be True

True

In [6]:
m.evaluate('P & R',g) # True and False should be False

False

In [7]:
m.evaluate('P | R',g) # True or False should be True

True

In [8]:
m.evaluate('-P | R',g) # not(True) or False should be False

False

In [9]:
m.evaluate('(P & Q) & (-Q | R)',g) #(True and True) and (not(True) or False) should be False

False

An expression involving the **equivalence** operator (<->) evaluates to True just when the Truth value of both operands are the same (both True, or both False)

In [10]:
m.evaluate('P <-> P',g) #X <-> X should always be True

True

In [11]:
m.evaluate('P <-> -P',g) #X <-> -X should always be False

False

In [12]:
m.evaluate('P <-> Q',g) #Will evaluate to True only if P == Q

True

The **implication** operator -> is very important but also the most confusing. Below is a table which shows the truth value of P -> Q for all combinations of P and Q


|P |Q |P->Q|
|--|--|--|
|True|True|True|
|False|True|True|
|False|False|True|
|True|False|False|

That is, an expression with the implication operator evaluates only False if the part before the arrow (the antecedent) is True but the conclusion (the proposition after the arrow) is False. If the antecedent is False, an implication is (trivially) True.  This may seem intuitively strange, but the reasoning is that if the antecedent is False, then we have no real information about the implication - we'd rather use other information to disprove the implication.

> This is actually kind of confusing. But the way I think about the case in row 2,3 is that if given a FALSE statement, basically we don't have any information about the next statement. Therefore we can only get True for the implication.

In [13]:
m.evaluate('P -> Q' ,g) #P = True, Q = True; True -> True == True

True

In [14]:
m.evaluate('R -> Q' ,g) #R = False, Q = True; False -> True == True

True

In [15]:
m.evaluate('P -> R' ,g) #P = True, R = False; True -> False == False

False

As it happens, we don't actually need all these operators, for instance the implication operator can be expressed **in terms of - and |**. (Implication evaluates to True if either the antecedent is False, or the conclusion is True).

> The antecedent is False, or the conclusion is True.

In [21]:
m.evaluate('-R | Q' ,g)

True

In [16]:
m.evaluate('R -> Q', g)

True

## De Morgan's Laws 

The negation of complex statements follows a set of rules commonly called "De Morgan's laws".  Simply put, the negation of an "AND" is an "OR" of the negation of each element, and the negation of an "OR" is an "AND" of the negation of each element:

-(A & B) <-> -A | -B <br>
-(A | B) <-> -A & -B

Let's test this: A = True; B = False.

A & B = False; -A | -B = True <br>
A | B = True; -A & -B = False <br>

## First-order logic

[First-order logic](https://en.wikipedia.org/wiki/First-order_logic) (FOL, also referred to as predicate logic or predicate calculus) is a much more powerful system of logic. In FOL, a *model* consists of a collection of *objects* or *entities* (known collectively as the *domain*). In NTLK's version of FOL, the domain is just represented as a set of strings.

In [18]:
#Let's create a cast of characters - these "people" will live in our domain.
dom = {"Arthur","Barbara","Charlie","Donna","Ed"}

In the context of FOL formulas, *variables* (often donated as single lower-case letters, particularly `x`,`y`, and `z`) represent one or more objects in the context of logical formulas. In NLTK, creating an `Assignment` (a dictionary mapping from variables to entities) is often a necessary step in working with logical formulas.

In [19]:
#Assign 'x' to "Arthur", 'y' to "Barbara", and 'z' to Charlie; now, anytime we refer to "x", are
#talking about "Arthur", etc.
Assignment(dom,[("x","Arthur"),("y","Barbara"),("z","Charlie")])

{'x': 'Arthur', 'y': 'Barbara', 'z': 'Charlie'}

In [21]:
Assignment(dom,[("x","Donna"),("y","Donna"),("z","Donna"),("x","Ed")])
#Notice that re-assignment just overwrites the variable

{'x': 'Ed', 'y': 'Donna', 'z': 'Donna'}

In [22]:
Assignment(dom,[("x","Arthur"),("y","Fred")])
#Fred is not in the domain - what happens?

AssertionError: 'Fred' is not in the domain: {'Barbara', 'Arthur', 'Donna', 'Ed', 'Charlie'}

Though if you don't want to explicitly assign any variables you can create an empty assignment:

In [32]:
Assignment(dom)

{}

The power of FOL comes from the notion of a **predicate**. Predicates look exactly like function definitions/calls in a programming language. We can write them as strings and turn them into logical *expressions*.

In [33]:
from nltk.sem import Expression
read_expr = Expression.fromstring #ie, read in a string, and treat it as an expression

In [34]:
exp1 = read_expr("person(Arthur)") #Arthur is a person
exp2 = read_expr("person(Barbara)") #So is Barbara
exp3 = read_expr("motherOf(Barbara,Arthur)") #Barbara is the mother of Arthur
exp4 = read_expr("person(x)") #x is also a person

In [35]:
exp1.pred       # pred:predicates

<ConstantExpression person>

In [36]:
exp1.args     # args: argument

[<ConstantExpression Arthur>]

In [37]:
exp3.pred

<ConstantExpression motherOf>

In [38]:
exp3.args

[<ConstantExpression Barbara>, <ConstantExpression Arthur>]

In [39]:
exp4.args

[<IndividualVariableExpression x>]

Intiuitively, **predicates express the properties of and relationships among entities in the domain**. One-place predicates such as `person` (also known as *properties*) are defined as a set of entities.  Note that in FOL, properties have no inherent "meaning" other than a set of entities. As such, we can define a property by listing the entities which have this property. In NLTK, we do this by creating a Valuation.

In [41]:
val = Valuation([("person",{"Arthur","Barbara","Charlie","Donna","Ed"}), 
                 ("male", {"Arthur","Charlie","Ed"}),
                 ("female", {"Barbara","Donna"}),
                 ("motherOf", {("Barbara","Arthur")})])

print(val) #Note that it's just creating a dictionary (but note the weird tuples for the properties)

{'female': {('Barbara',), ('Donna',)},
 'male': {('Ed',), ('Arthur',), ('Charlie',)},
 'motherOf': {('Barbara', 'Arthur')},
 'person': {('Barbara',), ('Charlie',), ('Arthur',), ('Ed',), ('Donna',)}}


Notice we've also defined one two-argument predicate. Two argument predicates are nothing more than sets of pairs of entities, and usually define relationships between entities.

We can extract a domain directly from a Valuation, and use the two of them to build a full model:

In [42]:
dom = val.domain
for entity in dom: # needed to make the connection between entities in Valuation and entities from read_exp
    val[entity] = entity
m = Model(dom, val)
g = Assignment(dom)

We can now use the evaluate method with our new model, and apply the operators from propositional logic

In [43]:
m.evaluate("person(Ed)",g)

True

In [44]:
m.evaluate("female(Ed)",g)

False

In [45]:
m.evaluate("-female(Ed)",g)

True

In [46]:
m.evaluate("female(Barbara) | female(Ed)",g)

True

In [47]:
m.evaluate("female(Barbara) -> -male(Barbara)",g)

True

In [48]:
m.evaluate("-female(Barbara) -> male(Ed)",g)

True

Example: Let's create a model with at least one more person and one more family relationship among our people, and demonstrate that it works using evaluate.

In [49]:
val = Valuation([("person",{"Arthur","Barbara","Charlie","Donna","Ed","Frida"}), 
                 ("male", {"Arthur","Charlie","Ed"}),
                 ("female", {"Barbara","Donna","Frida"}),
                 ("motherOf", {("Barbara","Arthur")}),
                 ("sisterOf", {("Donna", "Frida"),("Frida","Donna")})])
                 
dom = val.domain
for entity in dom: # needed to make the connection between entities in Valuation and entities from read_exp
    val[entity] = entity
m = Model(dom, val)
g = Assignment(dom)
 
m.evaluate("sisterOf(Donna,Frida) & sisterOf(Frida,Donna)",g)

True

The other major functionality of FOL is related to the variables we introduced earlier. If we add variables to our expressions, we can then *quantify* them. There are two kinds of quantifiers in FOL: The universal quantifier and the existential quantifier. A universal quantifier ($\forall$, "all" in NLTK) is true when any possible assignment of entities to the variable which follows it results in a True statement  

In [50]:
m.evaluate("all x. person(x)",g) #Read as "For all items x, x is a person"

True

In [45]:
m.evaluate("all x. male(x)",g) #"For all items x, x is male"

False

In [46]:
m.evaluate("all x. (male(x) -> -male(x))",g) #"For all items x, if x is male, then x is not male"

False

In [47]:
m.evaluate("all x y. (motherOf(x,y) -> female(x))",g) #"For all items x and items y, if x is the mother of y, then x is female"

True

The existential quantifier ($\exists$, "exists") asserts that at least one possible assignment returns True.

In [51]:
m.evaluate("exists x. female(x)",g) #"There is some x in g that is female."

True

In [49]:
m.evaluate("-exists x y. (male(x) & motherOf(x,y))",g) #"There is no x and y such that x is male and the mother of y

True

In [55]:
m.evaluate("all x all y. (male(x) -> -motherOf(x,y))",g) #"There is no x and y such that x is male and the mother of y

True

In [56]:
m.evaluate("all x. exists y. motherOf(y,x)",g) #For every x in g, there is a y that is its mother

False

Exercise: Let's create expressions with quantifiers involving the predicates we defined above. They don't have to be true.

In [57]:
m.evaluate("all x y. (sisterOf(x,y) <-> sisterOf(y,x)) ",g) #If x is a sister of y, then y is a sister of x

True

One final element of FOL that often co-occurs with the existential operator are the equality operators. The inequality operator is often more useful. For example, we need it to evaluate whether there are any mothers of more than one child (in our domain).

In [58]:
nltk.equality_preds()

equality       	=
inequality     	!=


In [61]:
m.evaluate("exists x y z.(motherOf(x,y) & motherOf(x,z) & (y != z))",g) #There is an x, y, and z.  If x is the mother of y and z, then y is not equal to z

False

In addition to building a complete, explicit model and then checking the truth of particular phrases (as we have above), another use case for first order logic is *inference*, which involves deducing the truth or falsity of a *conclusion* from a series of logic statements which are known to be true, i.e. the *premises*. When doing inference, there is a model, but it is implicit. The most typical rule of inference is *modus ponens*, which simply deduces B from the premises A and A->B.

Remember that for implication, if A is True, then if A->B, then B will also be true.

In [62]:
prover = nltk.TableauProver()

p1 = read_expr('man(socrates)') #A: Socrates is a man.
p2 = read_expr('all x.(man(x) -> mortal(x))') #A->B: If x is a man, then x is mortal.
p3  = read_expr('mortal(socrates)') #Thus, since Socrates is a man, he must be mortal.

prover.prove(p3, [p1,p2])

True

## Representing sentential meaning with logic

First-order logic is a powerful enough system to represent most of the meaning of languages in a compact computer-readable form. It has the advantage that it can abstract away from variations in syntactic structure and lexical choice. Consider the following sentences:

*Barbara is the mother of Arthur* <br >
*Barbara is Arthur's mother* <br >
*Barbara's Arthur's mom*

We can safely represent all of these as a single FOL formula:

In [63]:
# my code here
read_expr("motherOf(Barbara, Arthur)")
# my code here

<ApplicationExpression motherOf(Barbara,Arthur)>

It also generally provides an unambigious meaning in cases where the language is unclear:

*A man saw an elephant in his pyjamas*

In [64]:
#my code here
read_expr("exists x y z. (man(x) & elephant(y) & pyjamas(z) & saw(x,y) & wearing(x,z) & owns(x,z))")
#There is a man x who saw an elephant y, and the man was wearing his own pyjamas z
#my code here

<ExistsExpression exists x y z.(man(x) & elephant(y) & pyjamas(z) & saw(x,y) & wearing(x,z) & owns(x,z))>

In [65]:
#my code here
read_expr("exists x y z. (man(x) & elephant(y) & pyjamas(z) & saw(x,y) & wearing(y,z) & owns(x,z))")
#There is a man x who saw an elephant y, and the elephant was wearing the man's pyjamas z
#my code here

<ExistsExpression exists x y z.(man(x) & elephant(y) & pyjamas(z) & saw(x,y) & wearing(y,z) & owns(x,z))>

This is true even when a syntactic representation might not distinguish the meaning:

*Timmy is a bad boy*

In [66]:
#my code here
read_expr("boy(Timmy) & naughty(Timmy)")
#Timmy is a boy, and Timmy is naughty
#my code here

<AndExpression (boy(Timmy) & naughty(Timmy))>

In [68]:
#my code here
read_expr("badBoy(Timmy)")
#Timmy is a bad boy (whatever that means)
#my code here

<ApplicationExpression badBoy(Timmy)>

It is fairly straightforward to represent general statements about the world in FOL:

*All boys and girls are children*.

*All boys have mothers, and at least some love their mothers*

*All bad boys which own an elephant are happy as long as the elephant isn't wearing pyjamas*

In [85]:
#my code here
read_expr("all x.((boy(x) | girl(x)) -> child(x))") #Note that although the sentence uses "and", we use "or"
#my code here            

<AllExpression all x.((boy(x) | girl(x)) -> child(x))>

In [86]:
#my code here
read_expr("(all x. (boy(x) -> exists y. motherOf(y,x))) & (exists x y.((boy(x) & motherOf(y,x) & love(x,y))))")
#my code here 

<AndExpression (all x.(boy(x) -> exists y.motherOf(y,x)) & exists x y.(boy(x) & motherOf(y,x) & love(x,y)))>

In [67]:
#my code here
read_expr("all x.(badBoy(x) & exists y.(elephant(y) & owns(x,y) & -exists z.(pyjamas(z) & wearing(y,z))) -> happy(x))")
#my code here 

<AllExpression all x.((badBoy(x) & exists y.(elephant(y) & owns(x,y) & -exists z.(pyjamas(z) & wearing(y,z)))) -> happy(x))>

Note that we don't have to explicitly use quantifiers words in English to get a meaning which involves quantification in FOL

*Bad boys love their mothers if they aren't naughty*

In [68]:
#my code here
read_expr("all x y.((badBoy(x) & MotherOf(y,x) & -naughty(x)) -> love(x,y))")
#my code here

<AllExpression all x y.((badBoy(x) & MotherOf(y,x) & -naughty(x)) -> love(x,y))>

Some things (like explicit numbers) are a bit awkward, but possible!

*There are two bad boys under the bed.*

In [88]:
#my code here
read_expr("exists x y z.(boy(x) & boy(y) & naughty(x) & naughty(y) & x != y & bed(z) & under(z,x) & under(z,y))")
#my code here   

<ExistsExpression exists x y z.(boy(x) & boy(y) & naughty(x) & naughty(y) & -(x = y) & bed(z) & under(z,x) & under(z,y))>

## Ontologies, Knowledge Bases, and the Semantic Web

![](https://upload.wikimedia.org/wikipedia/commons/a/a9/LOD_Cloud_2014-08.svg)

[Ontologies](https://en.wikipedia.org/wiki/Ontology_(information_science%29) define the concepts, properties, and relations associated with some domain of knowledge. In first-order logic terms, they typically define the space of possible predicates and also sometimes relationships among predicates that would, in FOL, be expressed with quantified variables (e.g. $\forall x  Boy(x) \implies Person(x)$) There are many ontologies for specific fields, for example in the biomedical domain you will find:

- [Disease ontology](https://en.wikipedia.org/wiki/Disease_Ontology)
- [Gene ontology](https://en.wikipedia.org/wiki/Gene_ontology)
- [Clinical terms ontology](https://en.wikipedia.org/wiki/SNOMED_CT)
- [Biological Pathway ontology](http://www.biopax.org/)
- [Systems Biology ontology](https://en.wikipedia.org/wiki/Systems_Biology_Ontology)
- [Protein ontology](https://proconsortium.org/)
- [Medical education ontology](https://en.wikipedia.org/wiki/TIME-ITEM)
- ...

Some ontologies just provide very general frameworks for representing information, potentially across domains. A few examples of this kind of ontology include:

- [BFO](https://en.wikipedia.org/wiki/Basic_Formal_Ontology)
- [SUMO](https://en.wikipedia.org/wiki/Suggested_Upper_Merged_Ontology)
- [UMBEL](https://en.wikipedia.org/wiki/UMBEL)

WordNet is also sometimes considered an "upper" ontology of this sort.

Other ontologies have associated [knowledge bases](https://en.wikipedia.org/wiki/Knowledge_base) where specific facts about entities are stored. Two examples of general ontologies that are also knowledge bases is [DBpedia](https://wiki.dbpedia.org/) and [YAGO](https://en.wikipedia.org/wiki/YAGO_(database%29), both of which contain millions of facts extracted from Wikipedia.

There is a growing consensus arounding building ontologies/knowledge bases that are compatible with the [Semantic Web](https://en.wikipedia.org/wiki/Semantic_Web) framework. Three key properties of these sorts of ontologies are:

- Concepts grounded using Uniform Resource Identifiers (URIs)
- Basic facts expressed using Resource Description Framework (RDF) triples
- Relations among concepts captured using the Web Ontology Language (OWL)

> Question: why do we need URI instead of particular IDs? - We are trying to share the information and make them on the website as much as possible.

We will explore these three proprties via a Python library for RDF, rdflib. Let's first look an example of an RDF triple:

There are three elements to an RDF triple: subject, predicate, and object. In the case above, all three are represented by a URI which is actually a web page, though it is also possible to have an object which is a *literal*, e.g. a number or string. 

Let's build our own RDF triples using rdflib. When we are adding RDF triples, we are actually creating a knowledge graph, since we can think of the triples as linking two entities with a directed, labeled edge (the predicate).

In [70]:
#import sys
#!{sys.executable} -m pip install rdflib
import rdflib

g = rdflib.Graph() #Create the RDF Graph

Next, we create entities and literals.

In [71]:
from rdflib import URIRef, Literal

Arthur = URIRef("http://example.org/people/Arthur") #Create a URIReference for each character
Barbara = URIRef("http://example.org/people/Barbara")
Charlie = URIRef("http://example.org/people/Charlie")

A_name = Literal('Arthur') #We're going to use literals for the objects
B_name = Literal('Barbara')
C_name = Literal('Charlie')

entity_name_pairs = [(Arthur,A_name),(Barbara,B_name),(Charlie,C_name)]

Now let's populate our knowlege graph using the *Person* type and *knows* and *name* predicates from the [FOAF](https://en.wikipedia.org/wiki/FOAF_(ontology)) ontology (used for social media connections), and the *type* predicate that is build into RDF. The RDF namespace contains the URI for these ontologies and so we don't have to specify them. We `add` the triple to the graph.

In [72]:
from rdflib.namespace import RDF, FOAF

for entity, name in entity_name_pairs:
    #my code here
    g.add((entity,RDF.type, FOAF.Person)) #This is telling our graph that all of our entities are of type "FOAF.Person"
    g.add((entity,FOAF.name,name)) #And that their FOAF.name (a special property of FOAF graphs) is the name
    for entity2, _ in entity_name_pairs:
        g.add((entity,FOAF.knows,entity2)) #FOAF defines a "knows" relationship - importantly, it's reciprocal
    
    #my code here

We can write the graph to a file then read it back in.

In [73]:
S = g.serialize("test.txt",format="nt")
f = open("test.txt")
print(f.read())
f.close()

<http://example.org/people/Arthur> <http://xmlns.com/foaf/0.1/name> "Arthur" .
<http://example.org/people/Charlie> <http://xmlns.com/foaf/0.1/knows> <http://example.org/people/Barbara> .
<http://example.org/people/Arthur> <http://xmlns.com/foaf/0.1/knows> <http://example.org/people/Barbara> .
<http://example.org/people/Charlie> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://xmlns.com/foaf/0.1/Person> .
<http://example.org/people/Charlie> <http://xmlns.com/foaf/0.1/name> "Charlie" .
<http://example.org/people/Arthur> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://xmlns.com/foaf/0.1/Person> .
<http://example.org/people/Arthur> <http://xmlns.com/foaf/0.1/knows> <http://example.org/people/Arthur> .
<http://example.org/people/Barbara> <http://xmlns.com/foaf/0.1/name> "Barbara" .
<http://example.org/people/Barbara> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://xmlns.com/foaf/0.1/Person> .
<http://example.org/people/Barbara> <http://xmlns.com/foaf/0.1/knows> 

c:\users\nicol\appdata\local\programs\python\python37\lib\site-packages\rdflib\plugins\serializers\nt.py:37: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  "NTSerializer always uses UTF-8 encoding. "


In [74]:
g2 = rdflib.Graph()
g2.parse("test.txt",format="nt")
print(len(g2))

15


As alluded to earlier, we can make statements about predicates themselves, to allow for FOL inference. Some of these are part of RDF Schema (RDFS), and others are included only in OWL (a more expansive framework). There are too many for us to discuss in detail, but here are some examples:

- RDFS.subClassOf (assert one class is a subclass of another)
- RDFS.domain/range (assert that a subject/object of a particular predicate must be of a particular class - domain specifies the subject, while range specifies the object)
- OWL.disjointWith (assert that two classes have no common members)
- OWL.equivalentClasses (assert that two classes have the same members)
- RDFS.subPropertyOf (assert that if relationship A holds between objects, B will too) 
- OWL.SymmetricProperty (assert that a relationship is symmetric. ie, if A knows B, then B knows A, as well.)
- OWL.TransitiveProperty (assert that a relationship is transitive. ie, if A is a friend of B, and B is a friend of C, then A is a friend of C)
- OWL.inverseOf (assert that two relationships are inverses of one another, e.g. parentOf/childOf)

Let's test out a couple of these "meta" predicates, and show that they can be used to add new facts to our knowledge base. We'll create a class "Boy" which is a subclass of Person, and assert that FOAF "knows" is symmetric (not exactly true, but...).

In [76]:
from rdflib.namespace import RDFS,OWL
#!{sys.executable} -m pip install owlrl
import owlrl #pip install owlrl

g3 = rdflib.Graph()
Boy = URIRef("http://example.org/people/boy")
g3.add((Arthur,RDF.type, Boy))
g3.add((Barbara,FOAF.knows,Arthur))

#my code here
g3.add((Boy, RDFS.subClassOf,FOAF.Person)) #Boy is a subclass of Person: all boys are people
g3.add((FOAF.knows, RDF.type, OWL.SymmetricProperty)) #Knows is symmetric - if A knows B, B knows A
#my code here

owlrl.DeductiveClosure(owlrl.CombinedClosure.RDFS_OWLRL_Semantics).expand(g3) #Find all links that need to be connected

for subj,pred,obj in g3:
    if "Arthur" in str(subj):
        print(subj,pred,obj)


http://example.org/people/Arthur http://www.w3.org/2002/07/owl#sameAs http://example.org/people/Arthur
http://example.org/people/Arthur http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://example.org/people/boy
http://example.org/people/Arthur http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://example.org/people/Arthur http://xmlns.com/foaf/0.1/knows http://example.org/people/Barbara
http://example.org/people/Arthur http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/foaf/0.1/Person
http://example.org/people/Arthur http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Thing


Example: Let's create parentOf and childOf relationships, use RDF/OWL to assert that they are inverses and that Barbara is the parent of Arthur, then derive that Arthur is childOf Barbara

In [77]:
g4 = rdflib.Graph()

#your code here
parentOf = URIRef("http://example.org/parentOf")
childOf = URIRef("http://example.org/childOf")
g4.add((Barbara,parentOf,Arthur))
g4.add((parentOf,OWL.inverseOf,childOf)) #childOf and parentOf are inverses: if A is a child of B, then B is the parent of A
#your code here

owlrl.DeductiveClosure(owlrl.CombinedClosure.RDFS_OWLRL_Semantics).expand(g4)

for subj,pred,obj in g4:
    if "Arthur" in str(subj):
        print(subj,pred,obj)

http://example.org/people/Arthur http://www.w3.org/2002/07/owl#sameAs http://example.org/people/Arthur
http://example.org/people/Arthur http://example.org/childOf http://example.org/people/Barbara
http://example.org/people/Arthur http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://example.org/people/Arthur http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Thing


The system provided by RDF+OWL is refered to generally as a *description logic*. It is less powerful than FOL, but is far more practical to do inference with.

## Advanced topics: Lambda calculus, representing events

Though it seems clear that much language can be expressed as logical formulas, it is perhaps less clear how these formulas can be built up compositionally from the individual words in a sentence. It is useful, in particular, to have a way of expressing predicates independent of particular instantiations and/or quantifications of those predicates. Lambda ($\lambda$) notation provides us with a means to do this. For example, we could express the word *tall* as  

$\lambda.x$ tall($x$)

The $\lambda$ here expresses the fact that this expression is to some extent incomplete, and is waiting to be bound to a particular entity. This is done by a process known as $\lambda$-reduction, and involves nothing more than replacing the $\lambda$ variable (here, $x$ when the expression is applied, to, say, an entity)

$\lambda.x$ tall($x$)(John)

resulting in

tall(John)

In NLTK, lambda is expressed using `\` 

In [78]:
read_expr(r'\x.tall(x)(john)').simplify()

<ApplicationExpression tall(john)>

Multiple lambdas are possible for predicates with multiple arguments, with the lambdas being resolved in order as they are applied.

In [79]:
read_expr(r'\x.\y.like(x,y)(john)(mary)').simplify()

<ApplicationExpression like(john,mary)>

In [80]:
read_expr(r'\x.\y.like(x,y)(john)').simplify()

<LambdaExpression \y.like(john,y)>

Here the lambda calculus is like creating our own predicate, but more important, is that we can hold our 

Lambda calculus is essential for representing the **partial semantics** of phrases like "likes Mary" which form constituents in a parse tree.

Basic FOL seems fine for expressing immutable facts that do not vary, but what things involving time? The trick is to introduce events as "objects" in the model, that can then be assigned properties. For instance, consider the sentence:

*After I devoured the sandwich, I saw the elephant*


We can express the seeing event as follows:

$\exists e_1$ Seeing($e_1$) & Seer($e_1$,Speaker) & Seen($e_1$,Elephant)

And the eating event:

$\exists e_2$ Eating($e_2$) & Eater($e_2$,Speaker) & Eaten($e_2$,Sandwich) & Manner($e_2$, fast)

And further assert:

After($e_2,e_1$)

This kind of representation of events has the advantage that it does not require us to specify all the potential arguments of a multi-argument predicate `eat`, but instead we can add (sometimes optionally) additional pieces of information about the event in the form of two place predications where the event entity is the first argument.